In [1]:
from sdv.demo import load_tabular_demo
import pandas as pd
import numpy as np
from sdv.tabular import TVAE
from sdv.constraints import OneHotEncoding

In [2]:
# data = load_tabular_demo('student_placements')
# data1 = pd.read_excel('Seattle_vv_21.xlsx')
# a = pd.read_excel('Processed_Seattle_vv_21_modifiedbyFDR.xlsx',index_col=0)
# a.to_csv('processed_seattle_vv_21.csv',encoding='utf-8')

data1 = pd.read_csv('processed_seattle_vv_21.csv')
data1 = data1.drop(["UNDERINFLY"],axis=1)
data1 = data1.drop(['UNDERINFLN'],axis=1)
data1 = pd.get_dummies(data1,columns=['Severity'],dtype=int)
# data1.dtypes
# data1.shape

In [4]:


# data = data1.sample(frac=0.3)
data = data1.sample(n=100)
# print(data.shape)
# print(type(data))  #DataFrame

# print(data.dtypes)

data.shape

(100, 52)

In [5]:


#这里是各种onehot的constraints
addr_type_constraint = OneHotEncoding(
    column_names=['ADDRTYPEAlley','ADDRTYPEBlock','ADDRTYPEIntersection']
)

collision_type_constraint = OneHotEncoding(
    column_names=['COLLISIONTYPEAngles', 'COLLISIONTYPEHead.On', 'COLLISIONTYPELeft.Turn','COLLISIONTYPEParked.Car', 'COLLISIONTYPERear.Ended', 'COLLISIONTYPERight.Turn', 'COLLISIONTYPESideswipe']
)

month_constraint = OneHotEncoding(
    column_names=['Month1', 'Month10', 'Month11', 'Month12', 'Month2', 'Month3', 'Month4', 'Month5','Month6', 'Month7', 'Month8', 'Month9']
)

time_in_day_constraint = OneHotEncoding(
    column_names=['Timeevening_peak','Timemorning_peak', 'Timenon_peak']
)

# underinf_type_constraint = OneHotEncoding(  # 不知道什么意思
#     column_names=['UNDERINFLN','UNDERINFLY']
# )

weather_condition_constraint = OneHotEncoding(
    column_names=['WEATHERBlowing.Sand.Dirt', 'WEATHERBlowing.Snow',
       'WEATHERClear', 'WEATHERFog.Smog.Smoke', 'WEATHEROvercast',
       'WEATHERPartly.Cloudy', 'WEATHERRaining', 'WEATHERSevere.Crosswind',
       'WEATHERSleet.Hail.Freezing.Rain', 'WEATHERSnowing']
)

road_condition_constraint = OneHotEncoding(
    column_names=['ROADCONDDry',
       'ROADCONDIce', 'ROADCONDOil', 'ROADCONDSand.Mud.Dirt',
       'ROADCONDSnow.Slush', 'ROADCONDStanding.Water', 'ROADCONDWet']
)

light_condition_constraint = OneHotEncoding(
    column_names=['LIGHTCONDDark...No.Street.Lights',
       'LIGHTCONDDark...Street.Lights.Off', 'LIGHTCONDDark...Street.Lights.On',
       'LIGHTCONDDawn', 'LIGHTCONDDaylight', 'LIGHTCONDDusk']
)

severity_condition_constraint = OneHotEncoding(
    column_names=['Severity_Fatality Collision',
       'Severity_Injury Collision', 'Severity_Property Damage Only Collision',
       'Severity_Serious Injury Collision']
)



In [6]:
constraints = [
    month_constraint,
    addr_type_constraint,
    collision_type_constraint,
    
    time_in_day_constraint,
#     underinf_type_constraint,
    weather_condition_constraint,
    road_condition_constraint,
    light_condition_constraint,
    severity_condition_constraint
]
model = TVAE(constraints=constraints, enforce_min_max_values=False)

In [12]:
import time
import warnings
warnings.filterwarnings("ignore")
t1 = time.time()
model.fit(data)
t2 = time.time()
print('cost time = ',t2-t1)
model.save('./GANmodel/TVAE_model_use_100.pkl')

cost time =  5.798491954803467


In [14]:
load_model = TVAE.load('./GANmodel/TVAE_model_use_100.pkl')

In [17]:
generate_data_5k = load_model.sample(num_rows=5000)
generate_data_5k.shape

Sampling rows: 100%|████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 12530.95it/s]


(5000, 52)

In [18]:
generate_data_5000.to_csv('./GANmodel/data/100_to_5000.csv')

In [19]:
data.to_csv('./GANmodel/data/100.csv')

In [20]:
generate_data_10000 = load_model.sample(num_rows=10000)
print(generate_data_10000.shape)
generate_data_10000.to_csv('./GANmodel/data/100_to_10000.csv')

Sampling rows: 100%|██████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17823.51it/s]


(10000, 52)


In [21]:
generate_data_20000 = load_model.sample(num_rows=20000)
print(generate_data_20000.shape)
generate_data_20000.to_csv('./GANmodel/data/100_to_20000.csv')

Sampling rows: 100%|██████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 22604.97it/s]


(20000, 52)


In [9]:
from sdv.evaluation import evaluate
evaluate(new_data,data)

0.9448109983549045

In [10]:
data.sum()

ADDRTYPEAlley                                 9
ADDRTYPEBlock                              2163
ADDRTYPEIntersection                       1709
COLLISIONTYPEAngles                        1021
COLLISIONTYPEHead.On                         98
COLLISIONTYPELeft.Turn                      374
COLLISIONTYPEParked.Car                    1108
COLLISIONTYPERear.Ended                     758
COLLISIONTYPERight.Turn                      64
COLLISIONTYPESideswipe                      458
Month1                                      300
Month10                                     409
Month11                                      10
Month12                                       5
Month2                                      286
Month3                                      373
Month4                                      358
Month5                                      415
Month6                                      439
Month7                                      470
Month8                                  

In [11]:
new_data.sum()

ADDRTYPEAlley                                40
ADDRTYPEBlock                              5445
ADDRTYPEIntersection                       4515
COLLISIONTYPEAngles                        6324
COLLISIONTYPEHead.On                          0
COLLISIONTYPELeft.Turn                       70
COLLISIONTYPEParked.Car                    2320
COLLISIONTYPERear.Ended                    1137
COLLISIONTYPERight.Turn                       0
COLLISIONTYPESideswipe                      149
Month1                                     9083
Month10                                       0
Month11                                       0
Month12                                       0
Month2                                        0
Month3                                        0
Month4                                        0
Month5                                        0
Month6                                       96
Month7                                      821
Month8                                  

In [10]:
from sdv.evaluation import evaluate
evaluate(new_data,data)

0.9477444354151388

In [11]:
print(data.columns)

Index(['ADDRTYPEAlley', 'ADDRTYPEBlock', 'ADDRTYPEIntersection',
       'COLLISIONTYPEAngles', 'COLLISIONTYPEHead.On', 'COLLISIONTYPELeft.Turn',
       'COLLISIONTYPEParked.Car', 'COLLISIONTYPERear.Ended',
       'COLLISIONTYPERight.Turn', 'COLLISIONTYPESideswipe', 'Month1',
       'Month10', 'Month11', 'Month12', 'Month2', 'Month3', 'Month4', 'Month5',
       'Month6', 'Month7', 'Month8', 'Month9', 'Timeevening_peak',
       'Timemorning_peak', 'Timenon_peak', 'WEATHERBlowing.Sand.Dirt',
       'WEATHERBlowing.Snow', 'WEATHERClear', 'WEATHERFog.Smog.Smoke',
       'WEATHEROvercast', 'WEATHERPartly.Cloudy', 'WEATHERRaining',
       'WEATHERSevere.Crosswind', 'WEATHERSleet.Hail.Freezing.Rain',
       'WEATHERSnowing', 'ROADCONDDry', 'ROADCONDIce', 'ROADCONDOil',
       'ROADCONDSand.Mud.Dirt', 'ROADCONDSnow.Slush', 'ROADCONDStanding.Water',
       'ROADCONDWet', 'LIGHTCONDDark...No.Street.Lights',
       'LIGHTCONDDark...Street.Lights.Off', 'LIGHTCONDDark...Street.Lights.On',
       'LI

In [34]:
print(data['ADDRTYPEAlley'].sum()/len(data))
print(data['ADDRTYPEBlock'].sum()/len(data))
print(data['ADDRTYPEIntersection'].sum()/len(data))
print(50*'-')
print(new_data['ADDRTYPEAlley'].sum()/len(new_data))
print(new_data['ADDRTYPEBlock'].sum()/len(new_data))
print(new_data['ADDRTYPEIntersection'].sum()/len(new_data))

'''
使用全部数据

0.0023189899510435456
0.5573305849007988
0.4403504251481577
--------------------------------------------------
0.0056686421025508886
0.5485699561968564
0.44576140170059264

可以看出，ADDRTYPE  的分布还是基本符合

'''

'''
df = pd.DataFrame({"id":[1001,1002,1003,1004,1005,1006], 
 "date":pd.date_range('20130102', periods=6),
  "city":['Beijing ', 'SH', ' guangzhou ', 'Shenzhen', 'shanghai', 'BEIJING '],
 "age":[23,44,54,32,34,32],
 "category":['100-A','100-B','110-A','110-C','210-A','130-F'],
  "price":[1200,np.nan,2133,5433,np.nan,4432]},
  columns =['id','date','city','category','age','price'])

'''
# df = pd.DataFrame()
# df['ADDRTYPEAlley']=data['ADDRTYPEAlley'].sum()
# df['ADDRTYPEBlock']=data['ADDRTYPEBlock'].sum()
# df['ADDRTYPEIntersection']=data['ADDRTYPEIntersection'].sum()
# print(df)
# df.hist()

0.002577319587628866
0.5549828178694158
0.4424398625429553
--------------------------------------------------
0.006132440092759598
0.5909817057459418
0.40288585416129863


'\ndf = pd.DataFrame({"id":[1001,1002,1003,1004,1005,1006], \n "date":pd.date_range(\'20130102\', periods=6),\n  "city":[\'Beijing \', \'SH\', \' guangzhou \', \'Shenzhen\', \'shanghai\', \'BEIJING \'],\n "age":[23,44,54,32,34,32],\n "category":[\'100-A\',\'100-B\',\'110-A\',\'110-C\',\'210-A\',\'130-F\'],\n  "price":[1200,np.nan,2133,5433,np.nan,4432]},\n  columns =[\'id\',\'date\',\'city\',\'category\',\'age\',\'price\'])\n\n'

In [35]:
print(data['COLLISIONTYPEAngles'].sum()/len(data))
print(data['COLLISIONTYPEHead.On'].sum()/len(data))
print(data['COLLISIONTYPELeft.Turn'].sum()/len(data))
print(data['COLLISIONTYPEParked.Car'].sum()/len(data))
print(data['COLLISIONTYPERear.Ended'].sum()/len(data))
print(data['COLLISIONTYPERight.Turn'].sum()/len(data))
print(data['COLLISIONTYPESideswipe'].sum()/len(data))
print(50*'-')
print(new_data['COLLISIONTYPEAngles'].sum()/len(new_data))
print(new_data['COLLISIONTYPEHead.On'].sum()/len(new_data))
print(new_data['COLLISIONTYPELeft.Turn'].sum()/len(new_data))
print(new_data['COLLISIONTYPEParked.Car'].sum()/len(new_data))
print(new_data['COLLISIONTYPERear.Ended'].sum()/len(new_data))
print(new_data['COLLISIONTYPERight.Turn'].sum()/len(new_data))
print(new_data['COLLISIONTYPESideswipe'].sum()/len(new_data))

'''
使用全部数据

0.26307652666838444
0.02525122391136305
0.09636691574336512
0.28549342952847206
0.19531048698788972
0.01649059520742077
0.11801082195310487
--------------------------------------------------
0.5704715279567122
0.0
0.014171605256377222
0.22597268745168772
0.1499613501674826
0.0
0.03942282916774027

略有差距，但仍能接收

'''

0.27405498281786944
0.02577319587628866
0.08161512027491409
0.29381443298969073
0.18298969072164947
0.018900343642611683
0.12285223367697594
--------------------------------------------------
0.8988920381345015
0.0
0.0
0.10110796186549859
0.0
0.0
0.0


'\n使用全部数据\n\n0.26307652666838444\n0.02525122391136305\n0.09636691574336512\n0.28549342952847206\n0.19531048698788972\n0.01649059520742077\n0.11801082195310487\n--------------------------------------------------\n0.5704715279567122\n0.0\n0.014171605256377222\n0.22597268745168772\n0.1499613501674826\n0.0\n0.03942282916774027\n\n略有差距，但仍能接收\n\n'

In [199]:
print(data['Month1'].sum()/len(data))
print(data['Month2'].sum()/len(data))
print(data['Month3'].sum()/len(data))
print(data['Month4'].sum()/len(data))
print(data['Month5'].sum()/len(data))
print(data['Month6'].sum()/len(data))
print(data['Month7'].sum()/len(data))
print(data['Month8'].sum()/len(data))
print(data['Month9'].sum()/len(data))
print(data['Month10'].sum()/len(data))
print(data['Month11'].sum()/len(data))
print(data['Month12'].sum()/len(data))
print(50*'-')
print(new_data['Month1'].sum()/len(data))
print(new_data['Month2'].sum()/len(data))
print(new_data['Month3'].sum()/len(data))
print(new_data['Month4'].sum()/len(data))
print(new_data['Month5'].sum()/len(data))
print(new_data['Month6'].sum()/len(data))
print(new_data['Month7'].sum()/len(data))
print(new_data['Month8'].sum()/len(data))
print(new_data['Month9'].sum()/len(data))
print(new_data['Month10'].sum()/len(data))
print(new_data['Month11'].sum()/len(data))
print(new_data['Month12'].sum()/len(data))

'''
使用全部数据
0.07729966503478485
0.07369234733316156
0.09610925019324916
0.09224426694150992
0.10693120329811905
0.11311517650090183
0.12110280855449626
0.102035557845916
0.10821953104869879
0.10538520999742335
0.002576655501159495
0.0012883277505797476
--------------------------------------------------
0.8737438804431847
0.0
0.0
0.0
0.0
0.024220561710899254
0.102035557845916
0.0
0.0
0.0
0.0
0.0
有点难受

初步怀疑是向量维数过大？


'''

0.07729966503478485
0.07369234733316156
0.09610925019324916
0.09224426694150992
0.10693120329811905
0.11311517650090183
0.12110280855449626
0.102035557845916
0.10821953104869879
0.10538520999742335
0.002576655501159495
0.0012883277505797476
--------------------------------------------------
0.8737438804431847
0.0
0.0
0.0
0.0
0.024220561710899254
0.102035557845916
0.0
0.0
0.0
0.0
0.0


In [36]:
# 'Timeevening_peak','Timemorning_peak', 'Timenon_peak',
print(data['Timeevening_peak'].sum()/len(data))
print(data['Timemorning_peak'].sum()/len(data))
print(data['Timenon_peak'].sum()/len(data))
print(50*'-')
print(new_data['Timeevening_peak'].sum()/len(new_data))
print(new_data['Timemorning_peak'].sum()/len(new_data))
print(new_data['Timenon_peak'].sum()/len(new_data))

'''
使用全部数据

0.12316413295542386
0.07961865498582839
0.7972172120587477
--------------------------------------------------
0.1520226745684102
0.06415872197887143
0.7838186034527184

基本符合


'''

0.13144329896907217
0.08333333333333333
0.7852233676975945
--------------------------------------------------
0.0
0.0
1.0


'\n使用全部数据\n\n0.12316413295542386\n0.07961865498582839\n0.7972172120587477\n--------------------------------------------------\n0.1520226745684102\n0.06415872197887143\n0.7838186034527184\n\n基本符合\n\n\n'

In [37]:
'''
'WEATHERBlowing.Sand.Dirt',
       'WEATHERBlowing.Snow', 'WEATHERClear', 'WEATHERFog.Smog.Smoke',
       'WEATHEROvercast', 'WEATHERPartly.Cloudy', 'WEATHERRaining',
       'WEATHERSevere.Crosswind', 'WEATHERSleet.Hail.Freezing.Rain',
       'WEATHERSnowing',
'''
print(data['WEATHERBlowing.Sand.Dirt'].sum()/len(data))
print(data['WEATHERBlowing.Snow'].sum()/len(data))
print(data['WEATHERClear'].sum()/len(data))
print(data['WEATHERFog.Smog.Smoke'].sum()/len(data))
print(data['WEATHEROvercast'].sum()/len(data))
print(data['WEATHERPartly.Cloudy'].sum()/len(data))
print(data['WEATHERRaining'].sum()/len(data))
print(data['WEATHERSevere.Crosswind'].sum()/len(data))
print(data['WEATHERSleet.Hail.Freezing.Rain'].sum()/len(data))
print(data['WEATHERSnowing'].sum()/len(data))

print(50*'-')
print(new_data['WEATHERBlowing.Sand.Dirt'].sum()/len(new_data))
print(new_data['WEATHERBlowing.Snow'].sum()/len(new_data))
print(new_data['WEATHERClear'].sum()/len(new_data))
print(new_data['WEATHERFog.Smog.Smoke'].sum()/len(new_data))
print(new_data['WEATHEROvercast'].sum()/len(new_data))
print(new_data['WEATHERPartly.Cloudy'].sum()/len(new_data))
print(new_data['WEATHERRaining'].sum()/len(new_data))
print(new_data['WEATHERSevere.Crosswind'].sum()/len(new_data))
print(new_data['WEATHERSleet.Hail.Freezing.Rain'].sum()/len(new_data))
print(new_data['WEATHERSnowing'].sum()/len(new_data))

'''
使用全部数据
0.0
0.0
0.7088379283689771
0.002061324400927596
0.15949497552177275
0.0018036588508116465
0.12032981190414842
0.0002576655501159495
0.0002576655501159495
0.006956969853130637
--------------------------------------------------
0.04019582581808812
0.0
0.6565318216954393
0.0
0.21824272094820923
0.0
0.08502963153826333
0.0
0.0
0.0

我其实还好呜呜呜
'''


0.0
0.0
0.7173539518900344
0.000859106529209622
0.16323024054982818
0.0
0.11426116838487972
0.0
0.0
0.00429553264604811
--------------------------------------------------
0.20633857253285237
0.0
0.7402988920381345
0.0
0.009353259469208966
0.0
0.044009275959804174
0.0
0.0
0.0


'\n使用全部数据\n0.0\n0.0\n0.7088379283689771\n0.002061324400927596\n0.15949497552177275\n0.0018036588508116465\n0.12032981190414842\n0.0002576655501159495\n0.0002576655501159495\n0.006956969853130637\n--------------------------------------------------\n0.04019582581808812\n0.0\n0.6565318216954393\n0.0\n0.21824272094820923\n0.0\n0.08502963153826333\n0.0\n0.0\n0.0\n\n我其实还好呜呜呜\n'

In [38]:
'''
'ROADCONDDry', 'ROADCONDIce', 'ROADCONDOil',
       'ROADCONDSand.Mud.Dirt', 'ROADCONDSnow.Slush', 'ROADCONDStanding.Water',
       'ROADCONDWet',
'''

print(data['ROADCONDDry'].sum()/len(data))
print(data['ROADCONDIce'].sum()/len(data))
print(data['ROADCONDOil'].sum()/len(data))
print(data['ROADCONDSand.Mud.Dirt'].sum()/len(data))
print(data['ROADCONDSnow.Slush'].sum()/len(data))
print(data['ROADCONDStanding.Water'].sum()/len(data))
print(data['ROADCONDWet'].sum()/len(data))


print(50*'-')
print(new_data['ROADCONDDry'].sum()/len(new_data))
print(new_data['ROADCONDIce'].sum()/len(new_data))
print(new_data['ROADCONDOil'].sum()/len(new_data))
print(new_data['ROADCONDSand.Mud.Dirt'].sum()/len(new_data))
print(new_data['ROADCONDSnow.Slush'].sum()/len(new_data))
print(new_data['ROADCONDStanding.Water'].sum()/len(new_data))
print(new_data['ROADCONDWet'].sum()/len(new_data))

'''
使用全部数据
0.8080391651636176
0.0007729966503478485
0.0
0.0002576655501159495
0.006956969853130637
0.0002576655501159495
0.183715537232672
--------------------------------------------------
0.8281370780726617
0.0
0.0
0.0
0.0
0.0
0.17186292192733832

感觉还行

'''


0.8109965635738832
0.000859106529209622
0.0
0.0
0.003436426116838488
0.0
0.18470790378006874
--------------------------------------------------
0.8651893841793352
0.0
0.0
0.0
0.0
0.0
0.13481061582066478


'\n使用全部数据\n0.8080391651636176\n0.0007729966503478485\n0.0\n0.0002576655501159495\n0.006956969853130637\n0.0002576655501159495\n0.183715537232672\n--------------------------------------------------\n0.8281370780726617\n0.0\n0.0\n0.0\n0.0\n0.0\n0.17186292192733832\n\n感觉还行\n\n'

In [39]:
'''
'LIGHTCONDDark...No.Street.Lights',
       'LIGHTCONDDark...Street.Lights.Off', 'LIGHTCONDDark...Street.Lights.On',
       'LIGHTCONDDawn', 'LIGHTCONDDaylight', 'LIGHTCONDDusk',
'''

print(data['LIGHTCONDDark...No.Street.Lights'].sum()/len(data))
print(data['LIGHTCONDDark...Street.Lights.Off'].sum()/len(data))
print(data['LIGHTCONDDark...Street.Lights.On'].sum()/len(data))
print(data['LIGHTCONDDawn'].sum()/len(data))
print(data['LIGHTCONDDaylight'].sum()/len(data))
print(data['LIGHTCONDDusk'].sum()/len(data))



print(50*'-')
print(new_data['LIGHTCONDDark...No.Street.Lights'].sum()/len(new_data))
print(new_data['LIGHTCONDDark...Street.Lights.Off'].sum()/len(new_data))
print(new_data['LIGHTCONDDark...Street.Lights.On'].sum()/len(new_data))
print(new_data['LIGHTCONDDawn'].sum()/len(new_data))
print(new_data['LIGHTCONDDaylight'].sum()/len(new_data))
print(new_data['LIGHTCONDDusk'].sum()/len(new_data))

'''
使用全部数据
0.0056686421025508886
0.004380314351971141
0.23344498840505024
0.012110280855449627
0.7119299149703685
0.03246585931460964
--------------------------------------------------
0.07188868848234992
0.0
0.22313836640041226
0.0
0.7023962896160784
0.002576655501159495

感觉还行

'''



0.00859106529209622
0.005154639175257732
0.24484536082474226
0.012027491408934709
0.6915807560137457
0.037800687285223365
--------------------------------------------------
0.0984540066993043
0.0
0.021308940994589024
0.0
0.8802370523061067
0.0


'\n使用全部数据\n0.0056686421025508886\n0.004380314351971141\n0.23344498840505024\n0.012110280855449627\n0.7119299149703685\n0.03246585931460964\n--------------------------------------------------\n0.07188868848234992\n0.0\n0.22313836640041226\n0.0\n0.7023962896160784\n0.002576655501159495\n\n感觉还行\n\n'

In [207]:
'''
'Severity_Fatality Collision', 'Severity_Injury Collision',
       'Severity_Property Damage Only Collision',
       'Severity_Serious Injury Collision'
'''
print(data['Severity_Fatality Collision'].sum()/len(data))
print(data['Severity_Injury Collision'].sum()/len(data))
print(data['Severity_Property Damage Only Collision'].sum()/len(data))
print(data['Severity_Serious Injury Collision'].sum()/len(data))




print(50*'-')
print(new_data['Severity_Fatality Collision'].sum()/len(data))
print(new_data['Severity_Injury Collision'].sum()/len(data))
print(new_data['Severity_Property Damage Only Collision'].sum()/len(data))
print(new_data['Severity_Serious Injury Collision'].sum()/len(data))

'''
使用全部数据
0.000515331100231899
0.2934810615820665
0.6895130121102808
0.01649059520742077
--------------------------------------------------
0.03323885596495749
0.2638495233187323
0.7029116207163102
0.0

感觉能接受


'''


0.000515331100231899
0.2934810615820665
0.6895130121102808
0.01649059520742077
--------------------------------------------------
0.03323885596495749
0.2638495233187323
0.7029116207163102
0.0


In [10]:
data.sum()/len(data)

ADDRTYPEAlley                              0.002319
ADDRTYPEBlock                              0.557331
ADDRTYPEIntersection                       0.440350
COLLISIONTYPEAngles                        0.263077
COLLISIONTYPEHead.On                       0.025251
COLLISIONTYPELeft.Turn                     0.096367
COLLISIONTYPEParked.Car                    0.285493
COLLISIONTYPERear.Ended                    0.195310
COLLISIONTYPERight.Turn                    0.016491
COLLISIONTYPESideswipe                     0.118011
Month1                                     0.077300
Month10                                    0.105385
Month11                                    0.002577
Month12                                    0.001288
Month2                                     0.073692
Month3                                     0.096109
Month4                                     0.092244
Month5                                     0.106931
Month6                                     0.113115
Month7      

In [11]:
new_data.sum()/len(new_data)

ADDRTYPEAlley                              0.004896
ADDRTYPEBlock                              0.521257
ADDRTYPEIntersection                       0.473847
COLLISIONTYPEAngles                        0.603710
COLLISIONTYPEHead.On                       0.000000
COLLISIONTYPELeft.Turn                     0.005153
COLLISIONTYPEParked.Car                    0.212316
COLLISIONTYPERear.Ended                    0.129606
COLLISIONTYPERight.Turn                    0.000000
COLLISIONTYPESideswipe                     0.049214
Month1                                     0.899253
Month10                                    0.000000
Month11                                    0.000000
Month12                                    0.000000
Month2                                     0.000000
Month3                                     0.000000
Month4                                     0.000000
Month5                                     0.000000
Month6                                     0.001804
Month7      

In [12]:
(new_data.sum()/len(new_data)-data.sum()/len(data))/(data.sum()/len(data))

ADDRTYPEAlley                               1.111111
ADDRTYPEBlock                              -0.064725
ADDRTYPEIntersection                        0.076068
COLLISIONTYPEAngles                         1.294809
COLLISIONTYPEHead.On                       -1.000000
COLLISIONTYPELeft.Turn                     -0.946524
COLLISIONTYPEParked.Car                    -0.256318
COLLISIONTYPERear.Ended                    -0.336412
COLLISIONTYPERight.Turn                    -1.000000
COLLISIONTYPESideswipe                     -0.582969
Month1                                     10.633333
Month10                                    -1.000000
Month11                                    -1.000000
Month12                                    -1.000000
Month2                                     -1.000000
Month3                                     -1.000000
Month4                                     -1.000000
Month5                                     -1.000000
Month6                                     -0.